# Batch 1 Preprocessing
Group 4: Damiano Chini, Riccardo Gilmozzi, Gianmarco Piccinno & Alessandro Rizzuto
Useful links:
https://github.com/ComplexityBiosystems/obesity-score
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE2508

This code operates the preprocessing steps, namely (from the paper):

1) Probes containing missing values are excluded from the analysis.

2) Probes are mapped to Entrez ID labels if they are available in the associated platform.

3) Values corresponding to raw expression counts or gene expression intensity are log2 transformed (if necessary).

4) Probes mapping to the same Entrez ID label are averaged out.

5) Probes that cannot be mapped to a unique Entrez ID label are excluded from the analysis, as well as those that cannot be mapped to any Entrez ID label at all.

6) We apply a simple L 1 normalization in linear space, imposing that the sum of expression of all genes is constant among samples.

After these steps, each data set or batch is represented by a single expression matrix X. Each entry X i j represents the log 2 of the expression intensity of gene i in sample j.


In [2]:
import GEOparse
import pandas as pd
import numpy as np
from functools import *
import re

In [3]:
gse1 = GEOparse.get_GEO(filepath="./data/GSE2508_family.soft.gz")

11-Oct-2017 19:44:52 INFO GEOparse - Parsing ./data/GSE2508_family.soft.gz: 
11-Oct-2017 19:44:52 DEBUG GEOparse - DATABASE: GeoMiame
11-Oct-2017 19:44:52 DEBUG GEOparse - SERIES: GSE2508
11-Oct-2017 19:44:52 DEBUG GEOparse - PLATFORM: GPL91
11-Oct-2017 19:44:53 DEBUG GEOparse - PLATFORM: GPL92
11-Oct-2017 19:44:53 DEBUG GEOparse - PLATFORM: GPL93
11-Oct-2017 19:44:53 DEBUG GEOparse - PLATFORM: GPL94
11-Oct-2017 19:44:54 DEBUG GEOparse - PLATFORM: GPL95
11-Oct-2017 19:44:54 DEBUG GEOparse - PLATFORM: GPL8300
11-Oct-2017 19:44:54 DEBUG GEOparse - SAMPLE: GSM47224
11-Oct-2017 19:44:54 DEBUG GEOparse - SAMPLE: GSM47225
11-Oct-2017 19:44:54 DEBUG GEOparse - SAMPLE: GSM47226
11-Oct-2017 19:44:54 DEBUG GEOparse - SAMPLE: GSM47227
11-Oct-2017 19:44:54 DEBUG GEOparse - SAMPLE: GSM47228
11-Oct-2017 19:44:55 DEBUG GEOparse - SAMPLE: GSM47229
11-Oct-2017 19:44:55 DEBUG GEOparse - SAMPLE: GSM47230
11-Oct-2017 19:44:55 DEBUG GEOparse - SAMPLE: GSM47231
11-Oct-2017 19:44:55 DEBUG GEOparse - SAMPLE: 

In [4]:
plats_1 = list(gse1.gpls.keys())
print(plats_1)

['GPL91', 'GPL92', 'GPL93', 'GPL94', 'GPL95', 'GPL8300']


# Phenotype Table (Clinical Data)

In [5]:
samples1 = gse1.phenotype_data[["platform_id", "title"]]
sample1 = samples1.groupby(["platform_id"]); sample1.groups
d = {}                        
for l in plats_1:
    #print("\nPlatform: "+str(l)+"\n", sample1.get_group(l))
    #print("\nPlatform: "+str(l)+"\n", sample1.get_group(l)['title'])
    ls = "".join(list(sample1.get_group(l)['title']))
    lf = re.findall("Lean F", ls)
    of = re.findall("Obese F", ls)
    lm = re.findall("Lean M", ls)
    om = re.findall("Obese M", ls)
    #print("LF: ", len(lf), "\nOF: ", len(of), "\nLM: ", len(lm), "\nOM: ", len(om))
    d[l] = {"LF": len(lf), "OF": len(of), "LM": len(lm), "OM": len(om)}
#print(d)
df = pd.DataFrame(d); print(df)
df.sum(axis=1)
df.sum(axis=0)
print(samples1.head(), len(samples1))

    GPL8300  GPL91  GPL92  GPL93  GPL94  GPL95
LF        5      5     10     10     10     10
LM        5      5     10     10     10     10
OF        5      5     10     10     10     10
OM        5      4      9      9      9      9
         platform_id           title
GSM47224       GPL91  Lean F 01 subA
GSM47225       GPL91  Lean F 02 subA
GSM47226       GPL91  Lean F 03 subA
GSM47227       GPL91  Lean F 04 subA
GSM47228       GPL91  Lean F 05 subA 195


In [108]:
x = samples1['title'].apply(lambda x: x[:-len(x.split()[-1])].strip()).to_frame('samples')
#print(x.head(), type(x))
x['gender'] = x['samples'].map(lambda x: x.split(' ')[1])
x['cbmi'] = x['samples'].map(lambda x: x.split(' ')[0].lower())
#print(x)
#x.index = x['samples']; x.drop('samples', inplace=True, axis=1)
#print(x)
x = x.drop_duplicates()
x.index = x['samples']; x.drop('samples', inplace=True, axis=1)
print(x.shape)
print(x.head())


(39, 2)
          gender  cbmi
samples               
Lean F 01      F  lean
Lean F 02      F  lean
Lean F 03      F  lean
Lean F 04      F  lean
Lean F 05      F  lean


In [20]:
grouped = x.groupby(['samples'])
df = pd.DataFrame()
l = pd.DataFrame.from_dict(grouped.groups)
print(l.head(), '\n\n', l.shape)

  Lean F 01 Lean F 02 Lean F 03 Lean F 04 Lean F 05 Lean F 06 Lean F 07  \
0  GSM47224  GSM47225  GSM47226  GSM47227  GSM47228  GSM47229  GSM47230   
1  GSM47347  GSM47348  GSM47349  GSM47350  GSM47351  GSM47352  GSM47353   
2  GSM47386  GSM47387  GSM47388  GSM47389  GSM47390  GSM47391  GSM47392   
3  GSM47561  GSM47562  GSM47563  GSM47564  GSM47565  GSM47566  GSM47567   
4  GSM47823  GSM47824  GSM47825  GSM47826  GSM47827  GSM47828  GSM47829   

  Lean F 08 Lean F 09 Lean F 10    ...     Obese F 10 Obese M 01 Obese M 02  \
0  GSM47231  GSM47232  GSM47233    ...       GSM47328   GSM47329   GSM47330   
1  GSM47354  GSM47355  GSM47356    ...       GSM47376   GSM47377   GSM47378   
2  GSM47393  GSM47394  GSM47395    ...       GSM47415   GSM47416   GSM47417   
3  GSM47568  GSM47569  GSM47570    ...       GSM47590   GSM47591   GSM47592   
4  GSM47830  GSM47831  GSM47832    ...       GSM47852   GSM47853   GSM47854   

  Obese M 03 Obese M 04 Obese M 05 Obese M 06 Obese M 07 Obese M 08 Obese 

In [109]:
x.to_pickle('./output/batch1_pheno.p')
with open('./output/batch1_pheno.txt', 'w') as handle:
    x.to_csv(handle, sep='\t')

# Preprocessing of Expression Data (Batch 1)

Batch 1 is composed of five different datasets that use 5 different Affymetrix platforms, each one represents a technical replicate.

In [9]:
plats_1 = list(gse1.gpls.keys())
plats_1

['GPL91', 'GPL92', 'GPL93', 'GPL94', 'GPL95', 'GPL8300']

In [10]:
d = {}
samples1 = gse1.phenotype_data[["platform_id", "title"]]
sample1 = samples1.groupby(["platform_id"])

for plat in plats_1:
    d[plat] = gse1.pivot_samples('VALUE')[list(sample1.get_group(plat)[["title"]].index)]

#print(d['GPL95'].tail())

In [11]:
d_ann = {}
for key in d.keys():
    d_ann[key] = d[key].reset_index().merge(gse1.gpls[key].table[["ID", "ENTREZ_GENE_ID"]],
                                left_on='ID_REF', right_on="ID").set_index('ID_REF')
    d_ann[key].drop(['ID'], axis=1, inplace=True)
    print(d_ann[key].shape[0])


12626
12620
12646
12644
12639
12625


# 1) Probes containing missing values are excluded from the analysis.

In [12]:
for key in d_ann.keys():
    d_ann[key].dropna(inplace=True)
    print(str(key)+': ', d_ann[key].shape[0])

#print(d_ann['GPL92'].head())

GPL91:  12120
GPL92:  8766
GPL93:  7162
GPL94:  5018
GPL95:  7569
GPL8300:  12119


# 5) Probes that cannot be mapped to a unique Entrez ID label are excluded from the analysis, as well as those that cannot be mapped to any Entrez ID label at all.

In [13]:
for key in d_ann.keys():
    idx = d_ann[key].ENTREZ_GENE_ID.str.contains("///")#.index
    idx = idx[idx==True].index
    d_ann[key] = d_ann[key].drop(list(idx), axis=0)
    d_ann[key] = d_ann[key].groupby("ENTREZ_GENE_ID").mean()
    print(d_ann[key].head())
    print(str(key)+': ', d_ann[key].shape[0])

                   GSM47224     GSM47225     GSM47226  GSM47227     GSM47228  \
ENTREZ_GENE_ID                                                                 
10               378.000000   113.800000   164.500000     163.5   165.000000   
100             1340.950000  1232.950000  1464.750000    1048.8  1028.300000   
1000             575.150000   480.150000   327.400000     790.4   576.600000   
10000            196.100000   452.600000  1297.900000     873.1   642.800000   
10001            396.066667   518.266667   478.866667     557.0   535.933333   

                GSM47234  GSM47235    GSM47242     GSM47256     GSM47269  \
ENTREZ_GENE_ID                                                             
10                 82.30     56.80  112.200000   270.800000    68.200000   
100              1277.45   1432.35  551.300000   589.050000  1156.450000   
1000              847.55    443.70  237.000000   501.850000   479.350000   
10000             229.80    373.00  772.900000  1040.200000

# 2) Probes are mapped to Entrez ID labels if they are available in the associated platform.


# 3) Values corresponding to raw expression counts or gene expression intensity are log2 transformed (if necessary).

In [14]:
for key in d_ann.keys():
    d_ann[key] = np.log2(d_ann[key])
    print(str(key)+':\n', d_ann[key].head())
    

GPL91:
                  GSM47224   GSM47225   GSM47226   GSM47227   GSM47228  \
ENTREZ_GENE_ID                                                          
10               8.562242   6.830357   7.361944   7.353147   7.366322   
100             10.389040  10.267899  10.516439  10.034524  10.006046   
1000             9.167794   8.907341   8.354911   9.626439   9.171427   
10000            7.615446   8.822093  10.341964   9.770003   9.328226   
10001            8.629599   9.017551   8.903480   9.121534   9.065910   

                 GSM47234   GSM47235  GSM47242   GSM47256   GSM47269  \
ENTREZ_GENE_ID                                                         
10               6.362821   5.827819  6.809929   8.081084   6.091700   
100             10.319051  10.484168  9.106694   9.202246  10.175487   
1000             9.727155   8.793441  7.888743   8.971112   8.904936   
10000            7.844235   8.543032  9.594138  10.022645   9.811535   
10001            9.011507   8.806711  9.088612  

In [61]:
s = [d_ann[key] for key in d_ann.keys()]

In [38]:
df_final = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True, how='outer'), s)
print(df_final.head())
print(df_final.shape)

                 GSM47224   GSM47225   GSM47226   GSM47227   GSM47228  \
ENTREZ_GENE_ID                                                          
1                     NaN        NaN        NaN        NaN        NaN   
10               8.562242   6.830357   7.361944   7.353147   7.366322   
100             10.389040  10.267899  10.516439  10.034524  10.006046   
1000             9.167794   8.907341   8.354911   9.626439   9.171427   
10000            7.615446   8.822093  10.341964   9.770003   9.328226   

                 GSM47234   GSM47235  GSM47242   GSM47256   GSM47269  \
ENTREZ_GENE_ID                                                         
1                     NaN        NaN       NaN        NaN        NaN   
10               6.362821   5.827819  6.809929   8.081084   6.091700   
100             10.319051  10.484168  9.106694   9.202246  10.175487   
1000             9.727155   8.793441  7.888743   8.971112   8.904936   
10000            7.844235   8.543032  9.594138  10.02264

# Aggregate data about the 39 samples (tot=195)

In [22]:
print(l.head())

  Lean F 01 Lean F 02 Lean F 03 Lean F 04 Lean F 05 Lean F 06 Lean F 07  \
0  GSM47224  GSM47225  GSM47226  GSM47227  GSM47228  GSM47229  GSM47230   
1  GSM47347  GSM47348  GSM47349  GSM47350  GSM47351  GSM47352  GSM47353   
2  GSM47386  GSM47387  GSM47388  GSM47389  GSM47390  GSM47391  GSM47392   
3  GSM47561  GSM47562  GSM47563  GSM47564  GSM47565  GSM47566  GSM47567   
4  GSM47823  GSM47824  GSM47825  GSM47826  GSM47827  GSM47828  GSM47829   

  Lean F 08 Lean F 09 Lean F 10    ...     Obese F 10 Obese M 01 Obese M 02  \
0  GSM47231  GSM47232  GSM47233    ...       GSM47328   GSM47329   GSM47330   
1  GSM47354  GSM47355  GSM47356    ...       GSM47376   GSM47377   GSM47378   
2  GSM47393  GSM47394  GSM47395    ...       GSM47415   GSM47416   GSM47417   
3  GSM47568  GSM47569  GSM47570    ...       GSM47590   GSM47591   GSM47592   
4  GSM47830  GSM47831  GSM47832    ...       GSM47852   GSM47853   GSM47854   

  Obese M 03 Obese M 04 Obese M 05 Obese M 06 Obese M 07 Obese M 08 Obese 

In [66]:
df_c = {}

for column in l.columns:
    df_c[column] = df_final[list(l[column])]

for key in df_c.keys():
    df_c[key] = df_c[key].mean(axis=1)
    df_c[key] = df_c[key].to_frame()
    df_c[key] = df_c[key].rename(columns={0: key})
#print(df_c)

p = [df_c[key] for key in df_c.keys()]

#print(p)
df_f = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True, how='outer'), p)
print(df_f.shape)
print(df_f.head())

(18281, 39)
                Lean F 01  Lean F 02  Lean F 03  Lean F 04  Lean F 05  \
ENTREZ_GENE_ID                                                          
1               11.458509  10.838416  10.227135  11.093021  11.459995   
10               8.562242   6.830357   7.361944   7.353147   7.366322   
100             10.389040  10.267899  10.516439  10.034524  10.006046   
1000             9.167794   8.907341   8.354911   9.626439   9.171427   
10000            8.768243   9.501238   9.877079   9.550253   9.416446   

                Lean F 06  Lean F 07  Lean F 08  Lean F 09  Lean F 10  \
ENTREZ_GENE_ID                                                          
1               10.408224  10.526108  11.382408   9.840306   9.605294   
10               5.794416   6.312883   6.375039   6.590961   6.305606   
100             10.209819   9.453168   9.879736  10.269010   9.857670   
1000             8.270295   8.464954   8.309476   8.834629   9.180406   
10000            9.508978   9.330314  

# 4) Probes mapping to the same Entrez ID label are averaged out.

# Write the eprs4 dataframe in a text file

In [111]:
hn = df_f.T
hn.to_pickle('./output/batch1_exprs.p')
with open('./output/batch1_geno.txt', 'w') as handle:
    hn.to_csv(handle, sep='\t')

# Comparison with the data provided by the authors

In [19]:
or_data = pd.read_pickle('./data/GSE2508_geno.p')
print(or_data.shape, hn.shape)
print(len(set.intersection(set(or_data.columns), set(hn.columns))))

(39, 18491) (195, 18281)
18225


# 6) We apply a simple L 1 normalization in linear space, imposing that the sum of expression of all genes is constant among samples.